In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json

def get_sublinks(article_url):
  # Make a request to the Wikipedia article
  response = requests.get(article_url)

  # Parse the HTML of the Wikipedia article
  soup = BeautifulSoup(response.text, 'html.parser')

  # Find all the sublinks within the article
  sublinks = []
  for link in soup.find_all('a', href=True):
    # Check if the link is a sublink (begins with '/wiki/')
    if link['href'].startswith('/wiki/'):
      sublinks.append(link['href'])

  return sublinks  
  
sublinks = get_sublinks('https://en.wikipedia.org/wiki/Python_(programming_language)')

article_url = 'https://en.wikipedia.org/wiki/Python_(programming_language)'

# Get the name of the article from the URL
article_name = re.search(r'wiki/(.+?)$', article_url).group(1)
# Add a new column to the dataframe with the name of the article
art_array = sublinks


# Create a list of sublink dictionaries
sublinks_data = []
for sublink in sublinks:
  sublink_dict = {'sublink': sublink}
  sublinks_data.append(sublink_dict)

# Create a Pandas dataframe from the list of dictionaries
df = pd.DataFrame(sublinks_data)
df.columns = [article_name]
df['Python_(programming_language)'] = df['Python_(programming_language)'].apply(lambda x: x.replace('/wiki/', ''))
df = df[~df['Python_(programming_language)'].str.contains(r':\b')]
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df = df[~df['Python_(programming_language)'].str.contains(r'%{2}')]


def is_valid_article(url):
  # Make a request to the URL
  try:
    response = requests.get(url)
  except:
    return False

  # Check if the response is a valid Wikipedia article
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.find(id='firstHeading'):
      return True
  return False

# Add a new column to the dataframe indicating whether each sublink is a valid Wikipedia article
df['is_valid_article'] = df['Python_(programming_language)'].apply(lambda x: is_valid_article(f'https://en.wikipedia.org/wiki/{x}'))
df = df.drop(columns=['is_valid_article'])
df


